# Read in the data #

In [27]:
import pandas as pd
import numpy as np
import os
import re # regex
import shutil
import string
import nltk

# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_curve, confusion_matrix, auc, accuracy_score
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Model, Input, Sequential
from tensorflow.keras.callbacks import *
from datetime import datetime
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import plot_model

print(tf.__version__)

2.7.0


In [28]:
test_dataset = pd.read_json('./data/Sarcasm_Headlines_Dataset.json', lines=True)
test_dataset.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [29]:
train_dataset = pd.read_json('./data/Sarcasm_Headlines_Dataset_v2.json', lines=True)
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


# Basic data cleanup and column removal #

In [30]:
train_dataset = train_dataset.drop('article_link', axis=1)

In [31]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
dtypes: int64(1), object(1)
memory usage: 447.3+ KB


# Pre-processing #
1. contractions
2. stop words
3. lowercase
4. stemming
5. tokenize

Q: Do we need to do all of these?

### Contractions ###

In [32]:
def remove_contractions(sentence):
    sentence = re.sub(r"won\'t", "will not", sentence)
    sentence = re.sub(r"can\'t", "can not", sentence)
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence.lower()

In [33]:
train_dataset["headline"].apply(remove_contractions)
test_dataset["headline"].apply(remove_contractions)

0        former versace store clerk sues over secret 'b...
1        the 'roseanne' revival catches up to our thorn...
2        mom starting to fear son is web series closest...
3        boehner just wants wife to listen, not come up...
4        j.k. rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free-fall
26705                             america is best 20 hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie 2014
Name: headline, Length: 26709, dtype: object

In [34]:
y_train = train_dataset["is_sarcastic"]
y_test = test_dataset["is_sarcastic"]

In [35]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
dtypes: int64(1), object(1)
memory usage: 447.3+ KB


### Stop Words ###

In [36]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/azolyomi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### One way of doing it: ####

In [38]:
print(train_dataset["headline"][0])

thirtysomething scientists unveil doomsday clock of hair loss


## Tokenize ##

In [39]:
t = Tokenizer()
t.fit_on_texts(train_dataset["headline"])

encoded_train = t.texts_to_sequences(train_dataset["headline"])
encoded_test = t.texts_to_sequences(test_dataset["headline"])

max_length = 25

padded_train = pad_sequences(encoded_train, 
    maxlen = max_length, 
    padding = "post", 
    truncating = "post")

padded_test = pad_sequences(encoded_test, 
    maxlen = max_length, 
    padding = "post", 
    truncating = "post")

print(padded_train.shape, padded_test.shape, type(padded_train))

vocab_size = len(t.word_index) + 1
vocab_size

(28619, 25) (26709, 25) <class 'numpy.ndarray'>


30885

In [40]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

In [41]:
path_to_glove_file = "./glove/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [42]:
num_tokens = vocab_size + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 24737 words (6147 misses)


In [43]:
earlystop = EarlyStopping(monitor = "val_accuracy", 
                          patience = 7, 
                          verbose = 1,  
                          restore_best_weights = True, 
                          mode = 'max')

reduce_lr = ReduceLROnPlateau(monitor = "val_accuracy", 
                              factor = .4642,
                              patience = 3,
                              verbose = 1, 
                              min_delta = 0.001,
                              mode = 'max')

In [44]:
input = Input(shape = (max_length, ), name = "input")

embedding = Embedding(input_dim = vocab_size + 2, 
                      output_dim = 100, 
                      weights = [embedding_matrix], 
                      trainable = False)(input)

lstm = LSTM(32)(embedding)
flatten = Flatten()(lstm)

dense = Dense(16, activation = None, 
              kernel_initializer = "he_uniform")(flatten)

dropout = Dropout(.25)(dense)
activation = Activation("relu")(dropout)
output = Dense(2, activation = "softmax", name = "output")(activation)
model = Model(inputs = input, outputs = output)

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 25)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 25, 100)           3088700   
                                                                 
 lstm_2 (LSTM)               (None, 32)                17024     
                                                                 
 flatten_2 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 activation_2 (Activation)   (None, 16)                0   

In [45]:
model.fit(padded_train, y_train, 
        validation_data = (padded_test, y_test), 
        epochs = 12, 
        batch_size = 32)

Epoch 1/12


2022-01-24 16:06:34.845567: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-24 16:06:34.970206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  4/895 [..............................] - ETA: 20s - loss: 0.6931 - accuracy: 0.5000  

2022-01-24 16:06:35.060897: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


895/895 [==============================] - ETA: 0s - loss: 0.5825 - accuracy: 0.6862

2022-01-24 16:06:50.434173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-24 16:06:50.489862: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


895/895 [==============================] - 24s 26ms/step - loss: 0.5825 - accuracy: 0.6862 - val_loss: 0.4703 - val_accuracy: 0.7793
Epoch 2/12
895/895 [==============================] - 22s 25ms/step - loss: 0.4620 - accuracy: 0.7873 - val_loss: 0.4246 - val_accuracy: 0.8039
Epoch 3/12
895/895 [==============================] - 22s 25ms/step - loss: 0.4193 - accuracy: 0.8122 - val_loss: 0.3838 - val_accuracy: 0.8319
Epoch 4/12
895/895 [==============================] - 23s 25ms/step - loss: 0.3885 - accuracy: 0.8306 - val_loss: 0.3668 - val_accuracy: 0.8328
Epoch 5/12
895/895 [==============================] - 23s 25ms/step - loss: 0.3664 - accuracy: 0.8413 - val_loss: 0.3710 - val_accuracy: 0.8356
Epoch 6/12
895/895 [==============================] - 23s 25ms/step - loss: 0.3520 - accuracy: 0.8476 - val_loss: 0.3329 - val_accuracy: 0.8547
Epoch 7/12
895/895 [==============================] - 23s 25ms/step - loss: 0.3386 - accuracy: 0.8546 - val_loss: 0.3433 - val_accuracy: 0.8524
Epo

In [55]:
def standardize_singular(input_data):
    lowercase = tf.strings.lower(input_data)
    decontracted = remove_contractions(input_data)
    sequenced = t.texts_to_sequences([input_data])
    padded_sequenced = pad_sequences(
        sequenced,
        maxlen=max_length,
        padding = "post", 
        truncating = "post")
    return padded_sequenced

def standardize_map(input_array):
    return map(standardize_singular, input_array)
    

In [56]:
examples = [
    "Breakthrough Procedure Allows Surgeons To Transplant Pig Rib Directly Into Human Mouth",
    "Jan. 6 Committee Seeks Interview With Kevin McCarthy",
    "CDC Shortens COVID Isolation Guidelines to One Pump Up Song on Way to Work",
    "Report: Snickers Basically Protein Bar",
    "Crappy Music Has Helped Moron Through Hardest Times In His Pointless Life",
    "Is Eminem Leaving The Music Industry?",
    "Breaking News: Eminem Scared Out Of Music Industry By A Polar Bear",
    "Chancellor Yang praised for bold indecision",
    "Breaking News: UCSB shuts down permanently after Yang decides its too annoying to be a chancellor",
    "Munger Hall described as \"Absolutely Stunning\" by UCSB Chancellor",

]


standardized = standardize_map(examples)

prediction = model.predict(standardized)

for i in range(len(prediction)):
    if (prediction[i][1] >= 0.6):
        print("I'm pretty sure that's sarcastic...", prediction[i][1])
    else:
        print("I buy it!", prediction[i][1])

I'm pretty sure that's sarcastic... 0.6942241
I buy it! 0.01684583
I'm pretty sure that's sarcastic... 0.98533034
I'm pretty sure that's sarcastic... 0.9825166
I buy it! 0.1546543
I buy it! 0.010574713
I'm pretty sure that's sarcastic... 0.81611
I'm pretty sure that's sarcastic... 0.6766111
I'm pretty sure that's sarcastic... 0.9448826
I'm pretty sure that's sarcastic... 0.7626414


In [52]:
!mkdir -p saved_model
model.save('./saved_model/fake_news_v1')

INFO:tensorflow:Assets written to: ./saved_model/fake_news_v1/assets


INFO:tensorflow:Assets written to: ./saved_model/fake_news_v1/assets
